In [1]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
from data.hmm import CompositionalHMMDataset
from tasks.metalearn import MetaLearningTask
import jax
import jax.numpy as jnp
import jax.random as jr
import gymnasium as gym
import numpy as np
import tensorflow_probability.substrates.jax.distributions as tfd
import torch
from functools import partial

In [2]:
implicit_task = MetaLearningTask.load_from_checkpoint(
    "/network/scratch/l/leo.gagnon/latent_control_log/checkpoints/f818u4jn/last.ckpt", strict=False
).cuda()

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/pytorch/core/saving.py:191: Found keys that are in the model state dict but not in the checkpoint: ['seen_tokens', 'val_latents', 'train_latents']


In [3]:
metaenv = CompositionalHMMDataset(implicit_task.full_data.cfg)

In [4]:
ckpt = torch.load("/network/scratch/l/leo.gagnon/latent_control_log/checkpoints/f818u4jn/last.ckpt")
jnp.all(metaenv.latent_transmat ==  ckpt['dataset'].latent_transmat)

/tmp/ipykernel_1951456/350915153.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/network/scratch/l/leo.gagnon/latent_control_log/checkpoints/f818u4jn

Array(True, dtype=bool)

In [ ]:
metaenv.get_family_transition(0)

Array([0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 1., 1.], dtype=float16)

In [446]:
@jax.jit
def _step_in_cycle_family(family, hmm_index, state, key):
    next_state = tfd.Categorical(probs=metaenv.get_family_transition(hmm_index)[family, state]).sample(1, seed=key)
    next_obs = tfd.Categorical(probs=metaenv.get_emission(hmm_index)[next_state]).sample(1, seed=key)
    return next_state, next_obs

# step in a base cycle
@jax.jit
def _step_in_base_cycle(hmm_index, state, key):
    next_state = tfd.Categorical(probs=metaenv.get_base_transition(hmm_index)[state]).sample(1, seed=key)
    next_obs = tfd.Categorical(probs=metaenv.get_emission(hmm_index)[next_state]).sample(1, seed=key)
    return next_state, next_obs

# step in-place
@jax.jit
def _step_inplace(hmm_index, state, key):
    next_state = jnp.array([state])
    next_obs = tfd.Categorical(probs=metaenv.get_emission(hmm_index)[next_state]).sample(1, seed=key)
    return next_state, next_obs

# step in the hmm
@jax.jit
def _step_default(hmm_index, state, key):
    next_state = tfd.Categorical(probs=metaenv.get_transition(hmm_index)[state]).sample(1, seed=key)
    next_obs = tfd.Categorical(probs=metaenv.get_emission(hmm_index)[next_state]).sample(1, seed=key)
    return next_state, next_obs

In [123]:
_step_in_base_cycle(0,0,jr.PRNGKey(0))

(Array([6], dtype=int32), Array([[24]], dtype=int32))

In [124]:
_step_default(0,0,jr.PRNGKey(0))

(Array([17], dtype=int32), Array([[28]], dtype=int32))

In [492]:
_step_inplace(2,3,jr.PRNGKey(13))

(Array([3], dtype=int32), Array([[28]], dtype=int32))

In [451]:
next_state = jnp.array([1])

In [454]:
metaenv.get_emission(0)[0]

Array([5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06,
       5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06,
       5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06,
       5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-01, 5.e-06, 5.e-06, 5.e-06,
       5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-01, 5.e-06, 5.e-06,
       5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06, 5.e-06,
       5.e-06, 5.e-06], dtype=float16)

In [490]:
tfd.Categorical(probs=metaenv.get_emission(0)[1]).sample(1, seed=jr.PRNGKey(25))

Array([28], dtype=int32)

In [120]:
partial(_step_in_cycle_family, 0)(0, 0, jr.PRNGKey(3))

(Array([0], dtype=int32), Array([[37]], dtype=int32))

In [126]:
branches = [_step_default, _step_inplace, _step_in_base_cycle] + [
    partial(_step_in_cycle_family, i) for i in range(metaenv.cfg.cycle_families)
]

In [129]:
states = jnp.array([1,2,3])
hmms = jnp.array([123,0,10000])
actions = jnp.array([0,1,2])
keys = jr.split(jr.PRNGKey(0),3)

In [133]:
X,Z = jax.vmap(jax.lax.switch, [0, None, 0, 0, 0])(actions, branches, hmms, states, keys)

In [580]:
env.metaenv.get_transition(0).sum(1)

Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.], dtype=float16)

Array([1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
       0., 1., 1.], dtype=float16)

In [592]:
xd = jnp.fill_diagonal(metaenv.get_family_transition(0)[0], val=(1.0 - metaenv.get_family_transition(0)[0].sum(1)), inplace=False)

In [594]:
xd.sum(1)

Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.], dtype=float16)

In [810]:
metaenv.family_transmat.sum(0).sum(-1)[0,0][5]

Array(0., dtype=float16)

In [813]:
(1, *([1]*2), 4)

(1, 1, 1, 4)

In [5]:
class HMMEnv(gym.Env):

    def __init__(
        self,
        metaenv: CompositionalHMMDataset,
        ids: jnp.array,
        max_length: int,
        seed: int,
    ) -> None:
        super().__init__()
        self.metaenv = metaenv
        self.ids = ids
        self.max_length = max_length
        self.seed = seed

        self.states: jnp.array

        self.action_space = jnp

        init_prob = jax.vmap(self.metaenv.get_startprobs)(self.ids)
        self.init_prob_dist = tfd.Categorical(probs=init_prob)

        self.generator = np.random.default_rng(seed)

    def reset(self):
        self.states = self.init_prob_dist.sample(
            1,
            seed=jr.PRNGKey(
                self.generator.integers(
                    np.iinfo(np.int32).min, np.iinfo(np.int32).max, dtype=np.int32
                )
            ),
        )[0]

    # step in a cycle family
    @partial(jax.jit, static_argnames="self")
    def _step_in_cycle_family(self, family, hmm_index, state, key):    

        # Add a self loop when node is not in family
        transition = self.metaenv.get_family_transition(hmm_index)[family]
        transition = jnp.fill_diagonal(transition, val=(1.0 - transition.sum(1)), inplace=False)  

        next_state = tfd.Categorical(
            probs=transition[state]
        ).sample(1, seed=key)[0]
        next_obs = tfd.Categorical(
            probs=self.metaenv.get_emission(hmm_index)[next_state]
        ).sample(1, seed=key)[0]
        return next_state, next_obs

    # step in a base cycle
    @partial(jax.jit, static_argnames="self")
    def _step_in_base_cycle(self, hmm_index, state, key):
        next_state = tfd.Categorical(
            probs=self.metaenv.get_base_transition(hmm_index)[state]
        ).sample(1, seed=key)[0]
        next_obs = tfd.Categorical(
            probs=self.metaenv.get_emission(hmm_index)[next_state]
        ).sample(1, seed=key)[0]
        return next_state, next_obs

    # step in-place
    @partial(jax.jit, static_argnames="self")
    def _step_inplace(self, hmm_index, state, key):
        next_state = jnp.array([state])[0]
        next_obs = tfd.Categorical(
            probs=self.metaenv.get_emission(hmm_index)[next_state]
        ).sample(1, seed=key)[0]
        return next_state, next_obs

    # step in the hmm
    @partial(jax.jit, static_argnames="self")
    def _step_default(self, hmm_index, state, key):
        next_state = tfd.Categorical(
            probs=self.metaenv.get_transition(hmm_index)[state]
        ).sample(1, seed=key)[0]
        next_obs = tfd.Categorical(
            probs=self.metaenv.get_emission(hmm_index)[next_state]
        ).sample(1, seed=key)[0]
        return next_state, next_obs

    @partial(jax.jit, static_argnames="self")
    def _step(self, states, actions, keys):
        branches = [
            self._step_default,
            self._step_inplace,
            self._step_in_base_cycle,
        ] + [
            partial(self._step_in_cycle_family, i)
            for i in range(metaenv.cfg.cycle_families)
        ]
        return jax.vmap(jax.lax.switch, [0, None, 0, 0, 0])(
            actions, branches, self.ids, states, keys
        )

    def step(self, actions: jnp.array):

        keys = jr.split(
            key=jr.PRNGKey(
                self.generator.integers(
                    np.iinfo(np.int32).min, np.iinfo(np.int32).max, dtype=np.int32
                )
            ),
            num=len(actions),
        )

        self.states, obs = self._step(self.states, actions, keys)

        return obs

In [6]:
env = HMMEnv(metaenv=metaenv, ids=jnp.arange(100), max_length=0, seed=0)
env.reset()

In [60]:
#actions = jr.choice(jr.PRNGKey(np.random.randint(0,100000,)), a=6, shape=(100,))
actions = jnp.full(100, 4)
obs = env.step(actions)
env.states

Array([ 7, 12, 13,  5, 19,  3,  8,  5,  1,  1,  8, 11,  5,  9, 13,  8, 17,
       19, 12, 16,  5,  6,  8,  5, 13, 19,  2, 19,  5, 17,  4, 18, 12, 14,
       17,  4, 10,  4,  6, 13,  0,  7,  5, 19, 13,  7, 17,  6,  1, 17, 19,
       14, 18, 17, 10, 17,  3, 11,  2, 18,  0, 17, 10,  7,  4, 18, 16,  0,
        1,  8,  1, 12,  1, 15, 10, 15,  9, 18, 16, 15,  7, 10, 15,  7,  4,
       14,  8, 14,  6, 10,  9,  2, 15,  3,  8, 15, 10,  3,  2,  6],      dtype=int32)

In [56]:
env.states

Array([ 7, 12, 13,  5, 19,  8,  9,  5,  1,  1,  9, 11,  5,  0, 13,  9,  3,
       19, 12, 16,  5,  6,  9,  5, 13, 19,  2, 19,  5,  8,  4, 18, 12, 14,
        8,  4, 10,  4,  6, 13,  3,  7,  5, 19, 13,  7,  8,  6,  1,  0, 19,
       14, 18,  0, 10,  0, 17, 11,  2, 18,  9,  0, 10,  7,  4, 18, 16,  9,
        1,  3,  1, 12,  1, 15, 10, 15,  3, 18, 16, 15,  7, 10, 15,  7,  4,
       14, 17, 14,  6, 10,  3,  2, 15,  0, 17, 15, 10,  8,  2,  6],      dtype=int32)

In [385]:
env._step_default(0, 0, jr.PRNGKey(np.random.randint(10000)))

(Array(6, dtype=int32), Array(41, dtype=int32))